# F2PY: TOMS680 code for w(z)

A fortran implementation of the $w(z)$ function can be found here

 https://calgo.acm.org/
    
    file: 680.gz
    gams: C8a
    for: complex error function
    by: G.P.M. Poppe and C.M.J. Wijers
    ref: ACM TOMS 16 (1990) 47      

We could try to write a python translation by hand, but that would be tedious and error-prone.

What about using the `f2py` framework that is included with numpy?

  https://docs.scipy.org/doc/numpy/f2py
  
  https://www.numpy.org/devdocs/f2py/usage.html

In [24]:
%cd /home/bjackel/Dropbox/teach/physx81/sphinx/phys581/werf/toms680
# wget https://calgo.acm.org/680.gz
# gunzip, rename to toms680.f

/home/bjackel/Dropbox/teach/physx81/sphinx/phys581/werf/toms680


## First try

Compile the fortran source code to produce a module that can be called from python.  Apparently sucessful...

In [28]:
import subprocess
cmnd = r'/home/bjackel/anaconda3/bin/f2py'
cmnd += ' -c toms680.f'
cmnd += ' -m toms680'

print(cmnd)
subprocess.run( cmnd, shell=True, check=True)
#%sx 'f2py-c toms680.f -m toms680'

/home/bjackel/anaconda3/bin/f2py -c toms680.f -m toms680


CompletedProcess(args='/home/bjackel/anaconda3/bin/f2py -c toms680.f -m toms680', returncode=0)

The module loads without error, revealing a docstring and a single function with the expected calling convention.

In [41]:
import toms680
print('dir: ', dir(toms680) )
print('\n __doc__:', toms680.__doc__ )

dir:  ['__doc__', '__file__', '__loader__', '__name__', '__package__', '__spec__', '__version__', 'wofz']

 __doc__: This module 'toms680' is auto-generated with f2py (version:2).
Functions:
  wofz(xi,yi,u,v,flag)
.


In [43]:
help(toms680.wofz)
print('\n __doc__: ', toms680.wofz.__doc__)

Help on fortran object:

class fortran(object)
 |  Methods defined here:
 |  
 |  __call__(self, /, *args, **kwargs)
 |      Call self as a function.
 |  
 |  __repr__(self, /)
 |      Return repr(self).


 __doc__:  wofz(xi,yi,u,v,flag)

Wrapper for ``wofz``.

Parameters
----------
xi : input float
yi : input float
u : input float
v : input float
flag : input int



### Fail

For any input x,y the output u,v are always zero.  Maybe I should have read the documentation more carefully.

In [46]:
x, y = 1.0, 2.0
u, v = 0.0, 0.0
flag = False
toms680.wofz(x, y, u, v, flag)
print(flag, x, y, u, v)

False 1.0 2.0 0.0 0.0


We could make a signature file (.pyf) and edit to get the desired behavior.

In [48]:
import subprocess
cmnd = r'/home/bjackel/anaconda3/bin/f2py'
cmnd += ' toms680.f'
cmnd += ' -h toms680.pyf'
cmnd += ' -m toms680'
print(cmnd)
subprocess.run( cmnd, shell=True, check=True)

/home/bjackel/anaconda3/bin/f2py toms680.f -h toms680.pyf -m toms680


CompletedProcess(args='/home/bjackel/anaconda3/bin/f2py toms680.f -h toms680.pyf -m toms680', returncode=0)

In [ ]:
%edit toms680_f2py.f

In [49]:
import subprocess
cmnd = r'/home/bjackel/anaconda3/bin/f2py'
cmnd += ' -c toms680_f2py.f'
cmnd += ' -m toms680_f2py'
#cmnd = ' '.join(cmnd)
print(cmnd)
subprocess.run( cmnd, shell=True, check=True)

/home/bjackel/anaconda3/bin/f2py -c toms680_f2py.f -m toms680_f2py


CompletedProcess(args='/home/bjackel/anaconda3/bin/f2py -c toms680_f2py.f -m toms680_f2py', returncode=0)

In [50]:
import toms680_f2py as toms680
x, y = 1.0, 2.0
u, v = 0.0, 0.0
flag = False
toms680.wofz(x, y, u, v, flag)
print(flag, x, y, u, v)

TypeError: toms680_f2py.wofz() takes at most 2 arguments (5 given)

In [52]:
print('\n __doc__: ', toms680.wofz.__doc__)


 __doc__:  u,v,flag = wofz(xi,yi)

Wrapper for ``wofz``.

Parameters
----------
xi : input float
yi : input float

Returns
-------
u : float
v : float
flag : int



In [53]:
x, y = 1.0, 2.0
u, v = 0.0, 0.0
flag = False
u, v, flag = toms680.wofz(x, y)
print(flag, x, y, u, v)

0 1.0 2.0 0.21849261527489075 0.0929978093926019


In [58]:
import numpy as np
import scipy.special
print( scipy.special.wofz(np.complex(x,y)) )
print( scipy.special.wofz(np.complex(x,y)) - np.complex(u,v) )

(0.21849261527489067+0.09299780939260188j)
(-8.326672684688674e-17-1.3877787807814457e-17j)


### Victory!

Accuracy is comparable to "official" function and twice the speed !?

In [59]:
%timeit u, v, flag = toms680.wofz(x, y)
%timeit scipy.special.wofz(np.complex(x,y))

483 ns ± 10.6 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)
884 ns ± 5.61 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


## Under the hood

In [64]:
import subprocess
cmnd = r'/home/bjackel/anaconda3/bin/f2py'
cmnd += ' --debug-capi'
cmnd += ' -c toms680_f2py.f'
cmnd += ' -m toms680_f2py_debug'
print(cmnd)
subprocess.run( cmnd, shell=True, check=True)

/home/bjackel/anaconda3/bin/f2py --debug-capi -c toms680_f2py.f -m toms680_f2py_debug


CompletedProcess(args='/home/bjackel/anaconda3/bin/f2py --debug-capi -c toms680_f2py.f -m toms680_f2py_debug', returncode=0)

In [66]:
!nm -C -l toms680_f2py_debug.cpython-36m-x86_64-linux-gnu.so

00000000000032b0 T array_from_pyobj	/tmp/tmp3p6zprfn/src.linux-x86_64-3.6/fortranobject.c:654
0000000000207790 B __bss_start
0000000000207280 d capi_kwlist.14946
0000000000002780 t check_and_fix_dimensions	/tmp/tmp3p6zprfn/src.linux-x86_64-3.6/fortranobject.c:859
00000000002077a0 b completed.7696
0000000000004070 T copy_ND_array	/tmp/tmp3p6zprfn/src.linux-x86_64-3.6/fortranobject.c:1025
                 U cos@@GLIBC_2.2.5
                 w __cxa_finalize@@GLIBC_2.2.5
0000000000001c70 t deregister_tm_clones
00000000002071e0 d doc_f2py_rout_toms680_f2py_debug_wofz	/tmp/tmp3p6zprfn/src.linux-x86_64-3.6/toms680_f2py_debugmodule.c:149
0000000000001d00 t __do_global_dtors_aux
0000000000206d90 t __do_global_dtors_aux_fini_array_entry
0000000000001d50 t double_from_pyobj.part.0	/tmp/tmp3p6zprfn/src.linux-x86_64-3.6/toms680_f2py_debugmodule.c:90
00000000002071c0 d __dso_handle
0000000000206d98 d _DYNAMIC
0000000000207790 D _edata
00000000002077f8 B _end
                 U exp@@GLIBC_2.2.5
0000

In [2]:
%sx ls

['680_2py.f',
 '680.f',
 'Untitled.ipynb',
 'werf680.cpython-36m-x86_64-linux-gnu.so',
 'werf680_example.py',
 'werf680_smart.cpython-36m-x86_64-linux-gnu.so']

In [10]:
%sx /home/bjackel/anaconda3/bin/f2py -c 680.f -m werf680

['running build',
 'running config_cc',
 'unifing config_cc, config, build_clib, build_ext, build commands --compiler options',
 'running config_fc',
 'unifing config_fc, config, build_clib, build_ext, build commands --fcompiler options',
 'running build_src',
 'build_src',
 'building extension "werf680" sources',
 'f2py options: []',
 'f2py:> /tmp/tmpc5wok4v3/src.linux-x86_64-3.6/werf680module.c',
 'creating /tmp/tmpc5wok4v3/src.linux-x86_64-3.6',
 'Reading fortran codes...',
 "\tReading file '680.f' (format:fix,strict)",
 'Post-processing...',
 '\tBlock: werf680',
 '{}',
 'In: :werf680:680.f:wofz',
 'vars2fortran: No typespec for argument "xi".',
 '{}',
 'In: :werf680:680.f:wofz',
 'vars2fortran: No typespec for argument "yi".',
 '{}',
 'In: :werf680:680.f:wofz',
 'vars2fortran: No typespec for argument "u".',
 '{}',
 'In: :werf680:680.f:wofz',
 'vars2fortran: No typespec for argument "v".',
 '\t\t\tBlock: wofz',
 'Post-processing (stage 2)...',
 'Building modules...',
 '\tBuilding m